# Working with ATT&CK


# Background

pyattck can be used to query the ATT&CK TAXII server and interact with ATT&CK programatically. In this workbook, we show some basic usage of it, how to generate a coverage spreadsheet, and then how to generate a layer file for navigator.

* ATT&CK on Github
    * https://github.com/mitre/cti
    * https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json 

* pyattck
    * https://pyattck.readthedocs.io/en/latest/
    * https://swimlane.com/blog/swimlane-pyattack-works-with-mitre-att-ck-framework/

<hr>

## Part 1 - General usage

In [ ]:
from pyattck import Attck

attack = Attck()

If you're connecting through a web proxy, you should set the `HTTP_PROXY` and `HTTPS_PROXY` environment variables.

## Exploring Tactics

In [ ]:
for tactic in attack.tactics:
    print(tactic.name)

In [ ]:
example_tactic = attack.tactics[0]

In [ ]:
example_tactic.name

In [ ]:
print(example_tactic.description)

In [ ]:
for technique in example_tactic.techniques:
    print(technique.id, technique.name)

## Exploring Techniques

In [ ]:
example_technique = attack.techniques[1]

In [ ]:
example_technique.name

In [ ]:
example_technique.id

In [ ]:
example_technique.description

In [ ]:
example_technique.wiki

In [ ]:
for actor in example_technique.actors:
    print(actor.name)

In [ ]:
for mitigation in example_technique.mitigations:
    print(mitigation.name)

## Exploring Actors

In [ ]:
for actor in attack.actors:
    print(actor.name)

In [ ]:
for actor in attack.actors:
    print(actor.name)
    for technique in actor.techniques:
        print('-', technique.id, technique.name)

In [ ]:
for actor in attack.actors:
    if actor.name == 'APT28':
        print('Techniques\n====')
        for technique in actor.techniques:
            print('-', technique.id, technique.name)
        print('Tools\n=====')
        for tool in actor.tools:
            print(tool.name, '-', tool.description.replace('\n', ' '))
        print('\nMalware\n=====')
        for malware in actor.malwares:
            print(malware.name, '-', malware.description)

In [ ]:
from collections import Counter

mitigations_counter = Counter([
    mitigation.name
    for technique in attack.techniques
    for mitigation in technique.mitigations])

In [ ]:
mitigations_counter.most_common()

<hr>

# Part 2 - Generate Coverage Spreadsheet

## Group Techniques by Data Source

In [ ]:
from collections import defaultdict

def group_techniques_by_data_source(attack, actor_name=None):
    data_sources = defaultdict(list)
    for technique in attack.techniques:
        if actor_name:
            related_actors = [actor.name for actor in technique.actors]
            if actor_name not in related_actors:
                continue
        if technique.data_source:
            for data_source in technique.data_source:
                data_sources[data_source].append(technique.id)
    return data_sources

In [ ]:
data_source_map = group_techniques_by_data_source(attack)

In [ ]:
data_source_map.keys()

In [ ]:
data_source_map['Authentication logs']

In [ ]:
apt1_data_source_map = group_techniques_by_data_source(attack, actor_name='APT1')
apt1_data_source_map.keys()

In [ ]:
apt1_data_source_map['Authentication logs']

## Generate the spreadsheet

In [ ]:
import pandas as pd

def create_data_source_spreadsheet(data_source_map, fp='Data Source Coverage Spreadsheet.xlsx'):
    (pd.DataFrame([
        {'Data Source': data_source, 'Techniques': techniques}
         for data_source, techniques in data_source_map.items()])
     .assign(**{'Num of Techniques': lambda df: df.Techniques.str.len(),
                'Data Source Available?': '',
                'Comments': ''})
     .drop(columns=['Techniques'])
     .set_index('Data Source')
     .sort_index()
     .to_excel(fp))
    
    print(f'Wrote data source coverage spreadsheet to {fp}')

In [ ]:
create_data_source_spreadsheet(data_source_map)

In [ ]:
create_data_source_spreadsheet(apt1_data_source_map, fp='APT1 Data Source Coverage Spreadsheet.xlsx')

<hr>

# Part 3 - Generate Layer File

> ref: https://github.com/mitre-attack/attack-navigator/blob/master/layers/LAYERFORMATv2_2.md

In [ ]:
from pathlib import Path
from collections import defaultdict
import json


class NavigatorLayer(object):
    def __init__(self, version='2.2', name='Default Layer Name',
                 description='Default Description', domain='mitre-enterprise',
                 colors=['#ffffff', '#b3d9ff'], min_value=0, max_value=100,
                 sorting=0, view_mode=0, hide_disabled=False, stages=['act'], metadata=None,
                 platforms=['Windows', 'SaaS', 'Azure AD', 'Azure', 'GCP',
                            'Office 365', 'AWS', 'macOS', 'Linux']):
        self.content = {
            'name': name,
            'version': version,
            'domain': domain,
            'description': description,
            'filters': {
                'stages': stages,
                'platforms': platforms
            },
            'gradient': {
                'colors': colors,
                'maxValue': max_value,
                'minValue': min_value
            },
            'sorting': sorting,
            'viewMode': view_mode,
            'hideDisabled': hide_disabled,
            'metadata': [],
            'techniques': []}

        if metadata:
            self.content['metadata'].extend(metadata)

    def to_json(self, fp):
        data = json.dumps(self.content, indent=4)
        Path(fp).write_text(data)

In [ ]:
import pandas as pd


def populate_layer_from_spreadsheet(attack, actor_name=None, 
                                    spreadsheet_fp='Data Source Coverage Spreadsheet.xlsx'):
    layer_content = []
    spreadsheet = pd.read_excel(spreadsheet_fp)
    available_data_sources = spreadsheet[
        spreadsheet['Data Source Available?'].str.lower() == 'yes']

    covered_techniques = defaultdict(list)
    for technique in attack.techniques:
        
        if actor_name:
            related_actors = [actor.name for actor in technique.actors]
            if actor_name not in related_actors:
                layer_content.append({
                    'techniqueID': technique.id,
                    'enabled': False})
                continue
            
        for data_source in available_data_sources['Data Source'].unique():
            if technique.data_source and data_source in technique.data_source:
                covered_techniques[technique.id].append(data_source)

    for technique in covered_techniques:
        layer_content.append({
            'techniqueID': technique,
            'score': len(covered_techniques[technique]),
            'metadata': [{
                'name': 'Data Sources',
                'value': ', '.join(covered_techniques[technique])}]})

    return layer_content

In [ ]:
layer = NavigatorLayer(
    name='Data Source Coverage Map',
    description='Shows techniques where the organization has a relevant data source',
    max_value=1)

In [ ]:
layer.content['techniques'] = populate_layer_from_spreadsheet(
    attack, spreadsheet_fp='Data Source Coverage Spreadsheet.xlsx')

In [ ]:
layer.content

In [ ]:
layer.to_json('data_source_layer.json')

In [ ]:
apt1_layer = NavigatorLayer(
    name='APT1 Data Source Coverage Map',
    description='Shows techniques where the organization has a relevant data source to detect APT1',
    hide_disabled=True,
    max_value=1)

apt1_layer.content['techniques'] = populate_layer_from_spreadsheet(
    attack, actor_name='APT1', spreadsheet_fp='APT1 Data Source Coverage Spreadsheet.xlsx')

apt1_layer.to_json('APT1_layer.json')